<a href="https://colab.research.google.com/github/Llmortimer/character_morality/blob/main/PreprocessingSplitAudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!apt-get install -y espeak espeak-data libespeak1 libespeak-dev
!apt-get install -y --force-yes ffmpeg
!apt-get install -y --force-yes deb-multimedia-keyring
!apt-get install -y festival*
!apt-get install -y build-essential
!apt-get install -y flac libasound2-dev libsndfile1-dev vorbis-tools
!apt-get install -y libxml2-dev libxslt-dev zlib1g-dev
!apt-get install -y python-dev python-pip

!pip install numpy
!pip install ffmpeg-python
!pip install python-espeak
!pip install aeneas
!pip install ffmpy

!pip install matplotlib
!pip install simplejson
!pip install scipy
!pip install hmmlearn
!pip install eyeD3
!pip install pydub
!pip install scikit_learn
!pip install tqdm
!pip install plotly
!pip install pandas
!pip install pyAudioAnalysis
!pip install pickle5
!pip install cloudpickle
# !pip install noisereduce

Reading package lists... Done
Building dependency tree       
Reading state information... Done
espeak is already the newest version (1.48.04+dfsg-5).
espeak-data is already the newest version (1.48.04+dfsg-5).
libespeak-dev is already the newest version (1.48.04+dfsg-5).
libespeak1 is already the newest version (1.48.04+dfsg-5).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-470
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
W: --force-yes is deprecated, use one of the options starting with --allow instead.
Reading package lists... Done
B

In [21]:
from aeneas.executetask import ExecuteTask
from aeneas.task import Task
from ffmpy import FFmpeg
import re
# coding=utf-8    

from aeneas.exacttiming import TimeValue
from aeneas.language import Language
from aeneas.syncmap import SyncMapFormat
from aeneas.task import TaskConfiguration
from aeneas.textfile import TextFileFormat
from aeneas.runtimeconfiguration import RuntimeConfiguration
import aeneas.globalconstants as gc

from pyAudioAnalysis import audioBasicIO as aIO
from pyAudioAnalysis import audioSegmentation as aS
from pyAudioAnalysis import audioTrainTest as aT

import collections
import json

# import noisereduce as nr

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
input_video = "/content/drive/MyDrive/shared/Scripts_and_Films/IndianaLastCrusadeFilm.mp4"
output_ts = "IndianaLastCrusadeScenes.txt"

input_script = "/content/drive/MyDrive/shared/Scripts_and_Films/IndianaLastCrusadeScript.txt"
dialogue_script = "/content/drive/MyDrive/shared/Scripts_and_Films/IndianaLastCrusadeDialogue.txt"

output_audio = "/content/drive/MyDrive/shared/Scripts_and_Films/IndianaLastCrusadeAudio.mp4"
voice_audio = "/content/drive/MyDrive/shared/Scripts_and_Films/IndianaLastCrusadeAudioIsolated.mp3"

sync_map = "syncmap.jason"

In [19]:
#pre-process script by extracting only dialogue 
f = open(input_script).read()
matches = re.findall(r"\n\s+([A-Z ]+)\n *((.+\s)*)", f, re.M)
dialogue = []
characters = []
for i in matches:
    if '' not in i:
        dialogue.append(" ".join(i[1].split()) + "\n\n")
        characters.append(i[0])


with open(dialogue_script, "w") as ds:
    for inst in dialogue:
        ds.write(inst)

In [ ]:
# copy audio from video to seperate file
ff = FFmpeg(
    inputs = {input_video: None},
    outputs = {output_audio: '-vn -acodec copy'}
)
ff.cmd
ff.run()

In [44]:
#scene detection, for each frame assign a probablility that the scene has changed given the change of intensity of the pixels on the pervious frame.
#if this probablility is higher than 0.5 for a given frame the frame time, and probablilty have been output to a file "time.txt" and the frame itself output to a PNG in scenes/ 

import os
import subprocess

%ffmpeg -i /content/drive/MyDrive/shared/Scripts_and_Films/IndianaLastCrusadeFilm.mp4 -filter_complex "select=\'gt(scene,0.5)\',metadata=print:file=/content/drive/MyDrive/shared/scenes/time.txt" -vsync vfr /content/drive/MyDrive/shared/scenes/img%03d.png', shell=True)



1

In [38]:
# aT.file_classification(voice_audio, "/content/drive/MyDrive/shared/models/knn_sm","knn")
# [Fs, x] = aIO.read_audio_file(voice_audio)
# print(Fs)
# print(x)
# segments = aS.silence_removal(x, Fs, 0.020, 0.020, smooth_window = 1.0, weight = 0.3)
# print(segments)

[flagsInd, classesAll, acc, CM] = aS.mid_term_file_classification(voice_audio, "/content/drive/MyDrive/shared/svm_rbf_sm", "svm", True, '/content/drive/MyDrive/shared/Scripts_and_Films/segmented_audio.segments')


EOFError: ignored

In [ ]:
rconf = RuntimeConfiguration()

rconf[RuntimeConfiguration.MFCC_MASK_NONSPEECH] = True
rconf[RuntimeConfiguration.MFCC_MASK_NONSPEECH_L1] = True
# rconf[RuntimeConfiguration.MFCC_WINDOW_LENGTH_L1] = TimeValue(u"0.8")
# rconf[RuntimeConfiguration.MFCC_WINDOW_SHIFT_L1] = TimeValue(u"0.25")
# rconf[RuntimeConfiguration.MFCC_MASK_NONSPEECH_L2] = True
# rconf[RuntimeConfiguration.MFCC_MASK_NONSPEECH_L3] = True
rconf[RuntimeConfiguration.MFCC_MASK_LOG_ENERGY_THRESHOLD] = TimeValue(u"0.8")

rconf[RuntimeConfiguration.TTS] = 'festival'

rconf[RuntimeConfiguration.DTW_MARGIN] = 30

rconf[RuntimeConfiguration.VAD_MIN_NONSPEECH_LENGTH] = TimeValue(u"0.01")
# rconf[RuntimeConfiguration.VAD_LOG_ENERGY_THRESHOLD] = TimeValue(u"1")

config_string = u"task_language=eng|is_text_type=mplain|os_task_file_format=json|os_task_file_levels=1|is_audio_file_head_length=115|is_audio_file_tail_length=405|task_adjust_boundary_nonspeech_min=0.01|task_adjust_boundary_nonspeech_string=REMOVE"
#|task_adjust_boundary_algorithm=percent|task_adjust_boundary_percent_value=50
task = Task(config_string=config_string, rconf=rconf)
task.audio_file_path_absolute = voice_audio
task.text_file_path_absolute = dialogue_script
task.sync_map_file_path_absolute = sync_map

In [ ]:
ExecuteTask(task).execute()
print(task.sync_map)
task.output_sync_map_file()


NameError: name 'task' is not defined

In [ ]:
occurances = collections.Counter(characters)
print(occurances)

Counter({'INDY': 328, 'HENRY': 147, 'ELSA': 74, 'DONOVAN': 59, 'BRODY': 58, 'SALLAH': 28, 'VOGEL': 26, 'FEDORA': 12, 'KAZIM': 12, 'ROSCOE': 8, 'KNIGHT': 8, 'HERMAN': 5, 'PANAMA HAT': 5, 'SHERIFF': 5, 'BUTLER': 4, 'SULTAN': 4, 'HALFBREED': 3, 'SINISTER MAN': 3, 'NAZI SOLDIER': 3, 'MAN': 2, 'IRENE': 2, 'WOMAN': 2, 'OFFICER': 1, 'FIRST SCOUT': 1, 'SCOUTMASTER': 1, 'MEN': 1, 'ROUGH RIDER': 1, 'MALE STUDENT': 1, 'MAN IN CHAIR': 1, 'SECOND MAN': 1, 'LIEUTENANT': 1, 'RADIO OPERATOR': 1, 'GUARD': 1, 'FEMALE PASSENGER': 1, 'MALE PASSENGER': 1, 'PERISCOPE SOLDIER': 1})


In [ ]:
f = open("../Scripts_and_Films/syncmap.jason", "r")
data = json.load(f)


for count, dialogue in enumerate(data["fragments"]):
    print(characters[count] + ": " + dialogue["lines"][0] + " - duration: {:.3f}s -".format(float(dialogue["end"])-float(dialogue["begin"])) + " start: {:.3f}s -".format(float(dialogue["begin"]))  + " end: {:.3f}s".format(float(dialogue["end"])))

OFFICER: Dis-mount! - duration: 0.840s - start: 117.000s - end: 117.840s
FIRST SCOUT: Herman's horsesick! - duration: 1.920s - start: 117.840s - end: 119.760s
SCOUTMASTER: Chaps, don't anybody wander off. Some of the passageways in here can run for miles. - duration: 22.920s - start: 119.760s - end: 142.680s
HERMAN: I don't think this is such a good idea. - duration: 15.440s - start: 142.680s - end: 158.120s
HERMAN: What is it? - duration: 6.300s - start: 158.120s - end: 164.420s
FEDORA: Alfred, did you get anything yet? - duration: 26.160s - start: 165.240s - end: 191.400s
MAN: Nothing. Dig in. - duration: 2.360s - start: 191.400s - end: 193.760s
ROSCOE: Hey, I've got something! - duration: 5.800s - start: 193.760s - end: 199.560s
ROSCOE: I got something, Garth! - duration: 11.360s - start: 199.560s - end: 210.920s
ROSCOE: I got something... I got something right here. - duration: 7.480s - start: 210.920s - end: 218.400s
ROSCOE: (more whoops; then) Oh, look at that! (more whoops) We'r

In [ ]:
matches = []
with open("../Python_Scripts/time.txt") as file:
    for line in file:
        match = re.match(".*(?<=pts_time:)([0-9.]+)", line)
        if match:
            matches.append(match.group(1))

filmInfo = []
# for count, i in enumerate(matches):
#     if count == len(matches):
#         break
#     sceneInfo[count] = "{:.3f}".format(float(matches[count+1]) - float(i))
i=0
while i < len(matches)-1:
    sceneInfo = {}
    sceneInfo["id"] = i
    sceneInfo["duration"] = "{:.3f}".format(float(matches[i+1]) - float(matches[i]))
    sceneInfo["start"] = "{:.3f}".format(float(matches[i]))
    sceneInfo["end"] = "{:.3f}".format(float(matches[i+1]))
    filmInfo.append(sceneInfo)
    i+=1

for info in filmInfo:
    print(info)


{'id': 0, 'duration': '55.931', 'start': '66.900', 'end': '122.831'}
{'id': 1, 'duration': '6.507', 'start': '122.831', 'end': '129.338'}
{'id': 2, 'duration': '18.602', 'start': '129.338', 'end': '147.940'}
{'id': 3, 'duration': '183.475', 'start': '147.940', 'end': '331.415'}
{'id': 4, 'duration': '5.547', 'start': '331.415', 'end': '336.962'}
{'id': 5, 'duration': '20.312', 'start': '336.962', 'end': '357.274'}
{'id': 6, 'duration': '3.795', 'start': '357.274', 'end': '361.069'}
{'id': 7, 'duration': '1.377', 'start': '361.069', 'end': '362.446'}
{'id': 8, 'duration': '2.252', 'start': '362.446', 'end': '364.698'}
{'id': 9, 'duration': '17.976', 'start': '364.698', 'end': '382.674'}
{'id': 10, 'duration': '83.334', 'start': '382.674', 'end': '466.008'}
{'id': 11, 'duration': '3.336', 'start': '466.008', 'end': '469.344'}
{'id': 12, 'duration': '1.085', 'start': '469.344', 'end': '470.429'}
{'id': 13, 'duration': '1.168', 'start': '470.429', 'end': '471.597'}
{'id': 14, 'duration': '